In [1]:
#Import all necessary modules
import os
import sys
import json

In [2]:
#Reads x, y, z coordinates from the file
def ReadCoordinates(filename): #specify the full path of the .coord file to be read
    coordinate_list = []
    fulllist = []
    with open(filename,"r") as f:
        line_list = f.readlines()
        for line in line_list:
            if (line.isspace()): #x, y and z coordinate arrays are seperated by a blank space between rows
                fulllist.append(coordinate_list)
                coordinate_list = []
            else:
                for coord_i in line.split():
                    coordinate_list.append(float(coord_i))
        if (f.read() == ''): #indicates that we have reached the end of the file
            fulllist.append(coordinate_list)
    return fulllist

In [3]:
#Finds the units in which coordinate grid is defined
def ReadDistanceUnits(filename): #specify the full path of the grid_x/y/z.dat file to be read
    with open(filename,"r") as f:
        line_list = f.readlines()
        for word in line_list[0].split():
            if (word[:8] == 'Position'): #the distance unit is mentioned as Position(unit) in .dat file
                unit = word[-3:-1]
    return unit

In [4]:
#Reads 3D Potential data
def ReadPotential(filename): #specify the full path of the .dat file to be read
    Potential = []
    with open(filename,"r") as f:
        line_list = f.readlines()
        for line in line_list:
            for coord_i in line.split():
                Potential.append(coord_i)
    return Potential

In [5]:
#Reads control values, control names and voltage units for a run from .log file 
def ReadLog(filename, search_phrase = 'STARTING CALCULATION FOR BIAS POINT'): #specify the full path of the .log file to be read
    with open(filename,"r") as f:
        line_list = f.readlines()
        for i, line in enumerate(line_list):
            if (search_phrase in line):
                dump_in = [word for word in line_list[i + 1].split()]
                #the voltage for each gate is in the form ctrl_name, ctrl_value, unit 
                Control_names = [(dump_in[i]) for i in range (1, len(dump_in), 3)]
                Gate_voltages = [float(dump_in[i]) for i in range (2, len(dump_in), 3)]
                Voltage_unit = [(dump_in[i]) for i in range (3, len(dump_in), 3)]
                if (len(set(Voltage_unit)) == 1):  #checks if all gate voltages are given in same units or not
                    Voltage_unit = Voltage_unit[0]
    return tuple(Gate_voltages), Control_names, Voltage_unit

In [7]:
#The user would onl needs this function, the functions defined above are helper functions for SimulationReader
#Returns a dictionary that contains control names, control values, coordinates, potential data, distance units and potential units
def SimulationReader(directory): #specify the full path of the  directory which contains the simulation files
    DataDictionary = {} 
    Potential_data_Gate_voltages_run = {}
    GateVoltages = []
    for folders in os.scandir(directory):
        next_directory = os.path.join(directory, folders.name)
        for logfile in (os.listdir(next_directory)):
            if (logfile[-3:] == 'log'):
                logfile_path = os.path.join(next_directory, logfile)
                Gate_voltages, Control_names, Voltage_unit = ReadLog(logfile_path)
                GateVoltages.append(Gate_voltages)
        if ('output' in logfile):
            next_next_directory = os.path.join(next_directory, 'output')
            filename = os.path.join(next_next_directory, 'grid_x.dat') #assuming x, y, z coordinates are defined in same units
            if ((os.path.isfile(filename))):
                distance_units = ReadDistanceUnits(filename)
            for sub_subfolders in os.scandir(next_next_directory):
                if (sub_subfolders.name[:4] == 'bias' and sub_subfolders.name[-3:] != 'log'):
                    next_next_next_directory = os.path.join(next_next_directory, sub_subfolders.name)
                    for datafiles in os.listdir(next_next_next_directory):
                        if (datafiles == 'potential.coord'):
                            datafiles_path = os.path.join(next_next_next_directory, datafiles)
                            Coordinates = ReadCoordinates(datafiles_path) 
                        if (datafiles == 'potential.dat'):
                            datafiles_path = os.path.join(next_next_next_directory, datafiles)
                            Potential = ReadPotential(datafiles_path)
                            Potential_data_Gate_voltages_run[str(Gate_voltages)] = Potential #avoid str if we are not converting the dictionary to a json file
    #the coordinate grid and control names are same for all runs within a simulation, so store them only once
    DataDictionary = {'ControlNames' : Control_names, 'ControlValues' : GateVoltages, 'Coordinates' : Coordinates, 'PotentialData' : Potential_data_Gate_voltages_run, 'DistanceUnits' : distance_units, 'VoltageUnits' : Voltage_unit}
    #DataDictionary is in a format suitable to be directly converted to a .json file, its a regular python dictionary
    #Uncomment the next two lines and return json_loadedfile to enable conversion to a json file within the function, but this might create memory issues and may crash the program. Its safer to do the conversion outside the function
    #DataDictionary_Jsonfile = json.dumps(DataDictionary)
    #json_loadedfile = json.loads(DataDictionary_Jsonfile)
    return DataDictionary, #json_loadedfile